In [1]:
exec(open("inc_notebook.py").read())

current working dirF:\myProjects\tfKeras\UCSC\CMPS242\earthquake


In [2]:
import logging, sys, math,os
exec(open("estimator/initKeras.py").read())

Using TensorFlow backend.


In [3]:
from matplotlib import pyplot as plt
%matplotlib inline
import seaborn as sns
sns.set(style="darkgrid")

In [4]:
if sys.modules.get( 'estimator.EstimatorIO', False ) != False :
    del sys.modules['estimator.EstimatorIO'] 
if sys.modules.get( 'EstimatorIO', False ) != False :
    del sys.modules['EstimatorIO'] 
from estimator.EstimatorIO import *

logging.warning( "EstimatorIO loaded" )

estimatorIO = EstimatorIO()

if sys.modules.get( 'library', False ) != False :
    del sys.modules['library'] 
    
if sys.modules.get( 'library.OneStatsGeneratorForTestPos', False ) != False :
    del sys.modules['library.OneStatsGeneratorForTestPos'] 
if sys.modules.get( 'OneStatsGeneratorForTestPos', False ) != False :
    del sys.modules['OneStatsGeneratorForTestPos'] 
from library.OneStatsGeneratorForTestPos import *  
logging.warning( "OneStatsGeneratorForTestPos loaded" )

generator = OneStatsGeneratorForTestPos()


D:\ProgramData\Anaconda3\envs\tf\lib\site-packages\sklearn\base.py:251: UserWarning: Trying to unpickle estimator RobustScaler from version 0.20.2 when using version 0.20.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


In [5]:
trainModel = estimatorIO.loadTrainKerasModel('MixedFFN')
testModel = estimatorIO.loadTestKerasModel('MixedFFN')

In [6]:
import pandas as pd
output = pd.read_csv('F:/myProjects/cmps242/earthquake/data/sample_submission.csv')
dic = {}
for testName, batch in generator.batchesByFile():
#     print(f'test: {testName}')
#     print(trainModel.predict(batch, batch_size=generator.numberOfEmbeddingPerFile))
    pred = trainModel.predict(batch, batch_size=generator.numberOfEmbeddingPerFile)
    dic[testName] = pred
#     print( output.index[output['seg_id'] == testName])
    output.at[output.index[output['seg_id'] == testName], 'time_to_failure'] = np.median(pred)
    

In [7]:
output.to_csv('F:/myProjects/cmps242/earthquake/data/sample_submission-MixedFFN.csv', index=False)

In [8]:
testIds=[]
mean=[]
median=[]
maximum=[]
minimum=[]
for testName, predictions in dic.items():
    testIds.append(testName)
    mean.append(predictions.mean())
    median.append(np.median(predictions))
    maximum.append(np.max(predictions))
    minimum.append(np.min(predictions))


In [9]:
outputDf = pd.DataFrame({'seg_id': testIds, 'mean': mean, 'median': median, 'maximum': maximum, 'minimum': minimum})

In [10]:
outputDf.tail()

,seg_id,mean,median,maximum,minimum
2619,seg_ff4236,5.561367,5.392845,9.119740,1.359884
2620,seg_ff7478,5.722189,6.489578,9.462110,0.000000
2621,seg_ff79d9,4.449717,4.402524,9.408077,0.000000
2622,seg_ffbd6a,4.688509,4.204131,8.555380,0.000000
2623,seg_ffe7cc,7.940331,8.453941,9.292078,2.703769


In [11]:
outputDf.describe()

,mean,median,maximum,minimum
count,2624.000000,2624.000000,2624.000000,2624.000000
mean,5.809966,6.333000,8.968403,0.978355
std,1.280512,1.847928,0.523440,1.183612
min,0.965926,0.000000,3.748137,0.000000
25%,4.906227,5.003944,8.629330,0.000000
50%,5.747488,6.660600,9.021914,0.663805
75%,6.719123,7.867133,9.347435,1.644939
max,9.190258,9.635408,10.583025,7.971447


In [15]:
outputDf[outputDf['median'] < .1].count()

seg_id     3
mean       3
median     3
maximum    3
minimum    3
dtype: int64

In [16]:
outputDf[outputDf['mean'] < 2].count()

seg_id     6
mean       6
median     6
maximum    6
minimum    6
dtype: int64

In [17]:
outputDf[outputDf['maximum'] < .1].count()

seg_id     0
mean       0
median     0
maximum    0
minimum    0
dtype: int64

In [18]:
outputDf[outputDf['minimum'] < .1].count()

seg_id     1029
mean       1029
median     1029
maximum    1029
minimum    1029
dtype: int64

In [19]:
len(outputDf)

2624

In [20]:
output.head()

,seg_id,time_to_failure
0,seg_00030f,6.689303
1,seg_0012b5,7.537161
2,seg_00184e,6.724385
3,seg_003339,7.963380
4,seg_0042cc,6.618216


In [93]:
output.index[output['seg_id'] == 'seg_00184e']

Int64Index([2], dtype='int64')

In [87]:
output.at[2, 'time_to_failure'] = 100

In [88]:
output.iloc[2]

seg_id             seg_00184e
time_to_failure           100
Name: 2, dtype: object